In [15]:
%matplotlib inline
import json
from nltk.corpus import stopwords
from string import punctuation
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

In [16]:
# List of portuguese stopwords and punctuation
stopwords_pt = set(stopwords.words('portuguese') + list(punctuation))

d = list()

with open('../../data/sample.json', 'r') as f:
    for x in f:
        d.append(json.loads(x.replace('\n', '')))

# Load json to DataFrame
df = pd.DataFrame(d)

# Remove Null discourses
df = df.dropna(subset=['discourse'])

# Remove Null psrty
df = df.dropna(subset=['party'])

sample = df.sample(20000)

In [17]:
df.party = pd.Categorical(df.party)
df['target'] = df.party.cat.codes

In [18]:
features_train, features_test, labels_train, labels_test = train_test_split(df['discourse'], df['target'], test_size=0.3, random_state=42)

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words=stopwords.words('portuguese'), ngram_range=(1,3))
features_train_transformed = vectorizer.fit_transform(features_train)
features_test_transformed = vectorizer.transform(features_test)
selector = SelectPercentile(f_classif, percentile=10)
selector.fit(features_train_transformed, labels_train)
features_train_transformed = selector.transform(features_train_transformed).toarray()
features_test_transformed = selector.transform(features_test_transformed).toarray()

/Users/kureck/.lett/.virtualenvs/ml/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/Users/kureck/.lett/.virtualenvs/ml/lib/python3.5/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):
/Users/kureck/.lett/.virtualenvs/ml/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


In [6]:
# Bad bad performance
#from sklearn.svm import SVC
#from sklearn.metrics import accuracy_score

#clf = SVC(C=10000, kernel='rbf')
#clf.fit(features_train_transformed, labels_train)

#pred = clf.predict(features_test_transformed)

#print(accuracy_score(pred, labels_test))

In [13]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=58, random_state=5, min_samples_split=20)

clf.fit(features_train_transformed, labels_train)

pred = clf.predict(features_test_transformed)

print(accuracy_score(pred, labels_test))

0.8368858530162127


In [12]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

clf.fit(features_train_transformed, labels_train)

pred = clf.predict(features_test_transformed)

print(accuracy_score(pred, labels_test))

0.2749567936795326


In [29]:
f = pd.Series(["Bandido bom é bandido morto"], index=[0])

In [30]:
ftest = vectorizer.transform(f)

/Users/kureck/.lett/.virtualenvs/ml/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [31]:
ftest = selector.transform(ftest).toarray()

/Users/kureck/.lett/.virtualenvs/ml/lib/python3.5/site-packages/sklearn/utils/__init__.py:54: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


In [32]:
ftest.shape

(1, 11124)

In [33]:
p = clf.predict(ftest)

In [34]:
print(p)

[12]


In [21]:
p = clf.predict(ftest)

In [22]:
p

array([33], dtype=int8)